In [ ]:
# Add description here
#

In [ ]:
# Uncomment the next two lines to enable auto reloading for imported modules
# %load_ext autoreload
# %autoreload 2
# For more info, see:
# https://docs.ploomber.io/en/latest/user-guide/faq_index.html#auto-reloading-code-in-jupyter

In [2]:
# If this task has dependencies, declare them in the YAML spec and leave this
# as None
upstream = None

# This is a placeholder, leave it as None
product = None

In [ ]:
# Parameters
study1a1b_csv = "D:\\unibas\\new_embedding_risk_perception\\reanalysis\\data\\raw\\bhatia\\Study 1A and 1B - Risk Ratings.csv"
study2_csv = "D:\\unibas\\new_embedding_risk_perception\\reanalysis\\data\\raw\\bhatia\\Study 2 - Risk Ratings.csv"
product = {
    "nb": "D:\\unibas\\new_embedding_risk_perception\\risk_perception_bk\\pipeline\\reanalysis\\products\\process_risk.ipynb",
    "data": "D:\\unibas\\new_embedding_risk_perception\\risk_perception_bk\\pipeline\\reanalysis\\products\\risk_mean.csv",
}


In [71]:
import os
import pandas as pd

from scipy.stats import spearmanr
from sklearn.model_selection import ShuffleSplit

In [42]:
def load_bhatia_rawdata(filename:str, fieldpos:int):
    df_std1 = pd.read_csv(filename, header=1).iloc[1:]
    meta_cols = list(df_std1.columns[:18]) + list(df_std1.columns[-2:])
    risk_cols = [x for x in df_std1.columns if x not in meta_cols]
    df_risk = df_std1[risk_cols]
    new_cols = []
    for x in df_risk.columns:
        x = x.split('-')[fieldpos].strip()
        if x == 'waste inceneration':
            x = 'waste incineration'
        new_cols.append(x)
    df_risk.columns = new_cols
    
    return df_risk

In [43]:
dic_filename = {'study1a1b': {'filename':study1a1b_csv, 'fieldpos':0}, 'study2': {'filename':study2_csv, 'fieldpos':1}}
dt_raws = {key: load_bhatia_rawdata(**augments) for key, augments in dic_filename.items()}

In [109]:
df_raw = pd.concat(dt_raws.values()).astype(float)

In [95]:
def spearman_brown_coefficient(half_1, half_2):
    rho, _ = spearmanr(half_1, half_2)
    rho_sb = (2*rho)/(1+rho)
    return rho_sb

def calc_mean(ts1, rand_split):
    ts1 = ts1.dropna().reset_index(drop=True)
    idx1, idx2 = next(rand_split.split(ts1))
    return ts1[idx1].mean(), ts1[idx2].mean()
num_fold = 10

ts_rho_sb = []
for i in range(num_fold):
    rand_split = ShuffleSplit(n_splits=1, test_size=0.5, random_state=i)
    df_raw_split_mean = df_raw.apply(lambda ts1: calc_mean(ts1, rand_split))
    rho_sb = spearman_brown_coefficient(df_raw_split_mean.loc[0,:], df_raw_split_mean.loc[1,:])
    ts_rho_sb.append(rho_sb)

ts_rho_sb = pd.Series(ts_rho_sb)

# print mean and std of Spearman-Brown reliability coefficient
print('Mean of Spearman-Brown reliability coefficient: ', ts_rho_sb.mean())
print('Std of Spearman-Brown reliability coefficient: ', ts_rho_sb.std())

In [162]:
risk_mean = df_raw.mean(axis=0).reset_index()
risk_mean.columns = ['source', 'rating']

In [163]:
# Dropping 'Trump' from risk_mean since lowercase form that ends up getting pulled by SWOW has a very different meaning to uppercase form that is pulled by e.g. GloVe
risk_mean = risk_mean[risk_mean['source'] != 'Trump']

In [164]:
risk_mean.to_csv(product['data'], index=False)